In [2]:
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
import gc
import pandas as pd
from dateutil.parser import *
from transliterate import translit
"""pip install transliterate"""
import requests
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
import Levenshtein
"""pip install python-Levenshtein"""
import re
from io import StringIO
from calendar import monthrange
import yt.wrapper as yt
import json
from datetime import datetime, timedelta
import operator
import requests, os, sys, pymysql,time
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import aiohttp
import asyncio
import operator
import re
import pymysql
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/CloudSupport')

In [3]:
from startrek_client import Startrek
from joblib import Parallel, delayed

------------

# 1 часть
Считаем весь стафф

In [3]:
request = """
SELECT
    *
FROM "//home/cloud_analytics/import/staff/cloud_staff/cloud_staff"
FORMAT TabSeparatedWithNames
"""

In [4]:
staff = lib.execute_query(request)

Вторая линия

In [5]:
second_line = \
'Яндекс.Облако -> Отдел по развитию бизнеса и работе с заказчиками -> Направление клиентского опыта -> Группа профессиональных инженерных услуг'

In [6]:
def get_small_name(name):
    try:
        return name.split(" -> ")[1]
    except Exception:
        return name

def get_long_name(name):
    try:
        name.split(" -> ")[1]
        return ": ".join(name.split(" -> ")[1:])
    except Exception:
        return name

In [7]:
staff["short_name"] = staff["group_full_path"].apply(lambda name: get_small_name(name))

In [8]:
staff["short_name"].unique()

array(['Яндекс.Облако', 'Billing & Marketplace',
       'Machine Learning services', 'Development Platform',
       'Compute Cloud', 'Data platform', 'Virtual Private Cloud',
       'Отдел инфраструктуры Яндекс.Облака',
       'Cloud Security, Identity & Compliance', 'Compliance',
       'Key Management Service', 'Network Block Store',
       'Server Infrastructure', 'Internet of Things',
       'Отдел по развитию бизнеса и работе с заказчиками',
       'User Experience', 'Network Infrastructure',
       'IAM & Resource Management', 'Служба Маркетплейс Яндекс.Облака',
       'Hypervisor', 'Load Balancer'], dtype=object)

In [9]:
staff["long_name"] = staff["group_full_path"].apply(lambda name: get_long_name(name))

In [10]:
staff["is_second_line"] = (staff["group_full_path"] == second_line).astype(int)

Итого есть отдельно вторая линия, длинное имя, короткое имя

Теперь создадим словарь соответствий имен и их подразделений

In [11]:
support_name = 'Отдел по развитию бизнеса и работе с заказчиками'

In [12]:
short_group_dict = {} # name -> short_group
long_group_dict = {} # name -> long_group
for _, row in staff.iterrows():
    login = row["login"]
    if row["is_second_line"] == 0:
        short_group_dict[login] = row["short_name"]
        long_group_dict[login] = row["long_name"]
    else:
        short_group_dict[login] = "second line"
        long_group_dict[login] = "second line"

In [13]:
long_group_dict["tixoff"]

'Отдел по развитию бизнеса и работе с заказчиками: Направление клиентского опыта'

In [14]:
not_interseted_group = "Отдел по развитию бизнеса и работе с заказчиками"

In [15]:
second_line = "second line"

In [16]:
staff[staff["login"] == "datishin"]["group_full_path"].iloc[0]

'Яндекс.Облако -> Яндекс.Облако -> Отдел по развитию бизнеса и работе с заказчиками -> Отдел по развитию бизнеса и работе с заказчиками -> Направление по работе с ключевыми клиентами -> Направление по работе с ключевыми клиентами -> Направление по работе с ключевыми клиентами -> Направление по работе с ключевыми клиентами'

--------

## 1.1 часть

работа с events

In [17]:
class Event:
    def __init__(self, type, time, **kwargs):
        super().__init__()
        self.time = time
        if "timer starts" in type.lower():
            self.type = "timer starts"
            self.type_id = 1
        if "timer ends" in type.lower():
            self.type = "timer ends"
            self.type_id = 4
        if "status" in type.lower():
            self.type = "new status"
            self.type_id = 2
        if "призвали" in type.lower():
            self.type = "add summonee"
            self.type_id = 3
        self.args = []
        for key in kwargs:
            self.args.append(key)
            setattr(self, key, kwargs[key])
    
    def __repr__(self):
        ans_string = f"[event type: '{self.type}', start time: '{self.time}'"
        if len(self.args) > 0:
            added_args = ", ".join([key + ": '" + str(getattr(self, key)) + "'" for key in self.args])
            return ans_string + f", {added_args}]\n"
        else:
            return ans_string + "]\n"
    
    def __lt__(self, other):
        if self.time == other.time:
            return self.type_id < other.type_id
        return parse(self.time) < parse(other.time)

In [18]:
def added_summonee(changes, log):
    answers = []
    if "robot" not in log.updatedBy.id \
    and "яндекс" not in log.updatedBy.display.lower() \
    and "yandex" not in log.updatedBy.display.lower():
        if short_group_dict.get(log.updatedBy.id) == second_line:
            current_event = Event("призвали", log.updatedAt,
                                  user_id=log.updatedBy.id, 
                                  user=log.updatedBy.display)
            answers.append(current_event)

    if changes["field"].display == 'Нужен ответ пользователя':
        if changes["to"] is not None:
            for person in changes["to"]:
                current_event = Event("призвали", log.updatedAt,
                                      user_id=person.id, user=person.display)
                answers.append(current_event)
    if len(answers) > 0:
        return answers
    return None


def added_status(changes, log):
    ans_arr = []
    stop_statuses = ["Ждет выкладки", "Закрыт", "Линия 2", 
                     "Требуется информация", "Частично выполнен", 
                     "Подготовка документов", "Wait for Answer"]
    if changes["field"].display == 'Статус':
        curr_status = changes["to"].display
        ans_arr.append(Event("status changed", log.updatedAt, status=curr_status))
        if curr_status in stop_statuses:
            ans_arr.append(Event("timer ends", log.updatedAt, spent=0))
        if curr_status == "Открыт":
            ans_arr.append(Event("timer starts", log.updatedAt))
        return ans_arr
    return None

# 2 часть

Поработаем с трекером

Для начала возьмем названия всех интересующих тикетов

In [19]:
from startrek_client.settings import VERSION_SERVICE

In [20]:
client = Startrek(useragent="lunin-dv", 
                  base_url="https://st-api.yandex-team.ru/v2/myself", token=lib.startrek_token)

In [21]:
def get_ticket_name(issue):
    return issue._path.split("/")[-1]

Так же анализировать буду только закрытые тикеты, так как открытые могут еще меняться и неккоректно их использовать

In [99]:
def get_all_tickets():
    issues = client.issues.find(
        filter={'queue': 'CLOUDSUPPORT', "status":'closed'},
        order=['-created'],
        scrollType='unsorted',
        scrollTTLMillis=20000
    )
    ticket_names = []
    for issue in lib.tqdm(issues):
        ticket_names.append(get_ticket_name(issue))
    return ticket_names

In [100]:
ticket_names = get_all_tickets()

In [98]:
np.unique(ticket_names).shape[0]

22942

Теперь возьмем всю необходимую информацию про тикет.

In [25]:
def find_update_status_time(issue, log_list):
    answer_dict = {}
    next_time = None
    answer_dict["Открыт"] = issue.createdAt
    for log in log_list:
        used = False
        for changes in log.fields:
            if changes["field"].display == 'Статус':
                curr_status = changes["to"].display
                if curr_status is not None \
                and curr_status != "Открыт" and answer_dict.get("В работе") is None:
                    answer_dict["В работе"] = log.updatedAt
        if next_time is None and answer_dict["Открыт"] != log.updatedAt:
            next_time = log.updatedAt
    try:
        parse(answer_dict["В работе"])
    except Exception:
        answer_dict["В работе"] = next_time
    dt = parse(answer_dict["В работе"]) - parse(answer_dict["Открыт"])
    answer_dict["Начальный простой"] = dt.days * 24 * 60 + dt.seconds / 60
    return answer_dict

In [26]:
def get_issue_main_info(issue):
    answer_dict = {}
    answer_dict["pay type"] = issue.pay
    components_array = []
    for component in issue.components:
        components_array.append(component.display)
    answer_dict["components"] = components_array
    return answer_dict

In [88]:
def get_time_information(time_dict, issue, events):
    curr_start_time = None
    delta = 0
    answer_dict = {}
    deltas = []
    timer_events = []
    for ind, event in enumerate(events):
        if 'timer' in event.type:
            if len(timer_events) > 0 and\
            timer_events[-1].type == event.type and\
            event.type == 'timer starts':
                continue
            timer_events.append(event)

    for ind, event in enumerate(timer_events):
        if event.type == 'timer ends' and \
        (ind + 1 == len(timer_events) or timer_events[ind + 1].type != 'timer ends'):
            # последний закрывающий таймер
            dt = (parse(event.time) - parse(curr_start_time))
            delta = dt.days * 24 * 60 + dt.seconds / 60
            deltas.append(delta)
            event.spent = delta
            if len(deltas) == 1:
                event.spent -= time_dict["Начальный простой"]
        if event.type == 'timer starts':
            curr_start_time = event.time

    deltas[0] -= time_dict["Начальный простой"]
    answer_dict["all_active_time"] = sum(deltas)
    answer_dict["avg_active_time"] = np.mean(deltas)
    answer_dict["std_active_time"] = np.std(deltas)
    answer_dict["number_of_timer_stops"] = len(deltas) - 1
    answer_dict["percentile_25"], \
    answer_dict["percentile_50"], \
    answer_dict["percentile_75"] = np.percentile(deltas, [25, 50, 75], interpolation='midpoint').tolist()
    #print(timer_events)
    return answer_dict

In [89]:
def create_event_line(log_list, time_dict):
    events = []
    event_functions = [added_summonee, added_status]
    for log in log_list:
        for changes in log.fields:
            for func in event_functions:
                event = func(changes, log)
                if event is not None:
                    events += event
    events.append(Event("timer starts", time_dict["Открыт"]))
    return sorted(events)

In [90]:
def get_summonee_dict(time_dict, events):
    curr_last_time = time_dict["В работе"]
    delta = 0
    summonee_dict = {}
    groups = []
    is_started = False
    summonee_dict["help was needed"] = 0
    for event in events:
        if event.type == 'new status' and event.status == "В работе":
            is_started = True
        if event.type == 'timer ends' and is_started:
            delta += event.spent
        if event.type == 'timer starts' and is_started:
            curr_last_time = event.time

        if event.type == "add summonee" \
        and short_group_dict.get(event.user_id) is not None and \
        short_group_dict[event.user_id] != not_interseted_group:

            if summonee_dict.get("first summonee time") is None:
                
                dt = parse(event.time) - parse(curr_last_time)
                last_delta = dt.days * 24 * 60 + dt.seconds / 60
                delta += last_delta
                summonee_dict["help was needed"] = 1
                summonee_dict["first summonee time"] = delta
                summonee_dict["summonee short groups"] = set()
                summonee_dict["summonee long groups"] = set()
                # обнулили все
            try:
                summonee_dict["summonee short groups"].add(short_group_dict[event.user_id])
                summonee_dict["summonee long groups"].add(long_group_dict[event.user_id])
                if short_group_dict[event.user_id] != second_line:
                    summonee_dict["help from not second line"] = 1

            except Exception:
                pass
    return summonee_dict

In [91]:
def get_support_names(log_list):
    support_names = set()
    second_line_names = set()
    for log in log_list:
        for changes in log.fields:
            if "robot" not in log.updatedBy.id \
            and "яндекс" not in log.updatedBy.display.lower() \
            and "yandex" not in log.updatedBy.display.lower():
                try:
                    if short_group_dict[log.updatedBy.id] == not_interseted_group:
                        support_names.add(log.updatedBy.id)
                    if short_group_dict[log.updatedBy.id] == second_line:
                        second_line_names.add(log.updatedBy.id)
                except Exception:
                    pass
    help_from_support = 0
    if len(second_line_names) > 0:
        help_from_support = 1
    return {"support": support_names, "second line": second_line_names, 
            "support number": len(support_names), "second line number": len(second_line_names),
            "help from second line": help_from_support}

In [92]:
def get_all_ticket_info(ticket_name):
    try:
        issue = client.issues[ticket_name]
        log_list = list(issue.changelog.get_all(sort='asc'))
        time_dict = find_update_status_time(issue, log_list)
        main_ticket_info = get_issue_main_info(issue)
        events = create_event_line(log_list, time_dict)
        delta_info = get_time_information(time_dict, issue, events)

        support_dict = get_support_names(log_list)
        summonee_dict = get_summonee_dict(time_dict, events)
        answer_dict = {"ticket_name": ticket_name,
                       **main_ticket_info,
                       **delta_info,
                       **time_dict,
                       **summonee_dict,
                       **support_dict}
        #print(events)
        return answer_dict
    except Exception:
        print(ticket_name)
        raise

In [93]:
ticket_name = "CLOUDSUPPORT-9600"
get_all_ticket_info(ticket_name)

{'ticket_name': 'CLOUDSUPPORT-9600',
 'pay type': None,
 'components': ['other'],
 'all_active_time': 556.25,
 'avg_active_time': 556.25,
 'std_active_time': 0.0,
 'number_of_timer_stops': 0,
 'percentile_25': 556.25,
 'percentile_50': 556.25,
 'percentile_75': 556.25,
 'Открыт': '2019-07-31T20:10:56.154+0000',
 'В работе': '2019-07-31T22:01:57.813+0000',
 'Начальный простой': 111.01666666666667,
 'help was needed': 0,
 'support': {'amatol', 'trofogol'},
 'second line': set(),
 'support number': 2,
 'second line number': 0,
 'help from second line': 0}

In [94]:
ticket_name = 'CLOUDSUPPORT-29064'
get_all_ticket_info(ticket_name)

{'ticket_name': 'CLOUDSUPPORT-29064',
 'pay type': 'free',
 'components': [],
 'all_active_time': 0.0,
 'avg_active_time': 0.0,
 'std_active_time': 0.0,
 'number_of_timer_stops': 0,
 'percentile_25': 0.0,
 'percentile_50': 0.0,
 'percentile_75': 0.0,
 'Открыт': '2020-03-24T11:41:59.697+0000',
 'В работе': '2020-03-24T11:43:31.751+0000',
 'Начальный простой': 1.5333333333333334,
 'help was needed': 0,
 'support': set(),
 'second line': set(),
 'support number': 0,
 'second line number': 0,
 'help from second line': 0}

In [95]:
ticket_name = 'CLOUDSUPPORT-28018'
get_all_ticket_info(ticket_name)

{'ticket_name': 'CLOUDSUPPORT-28018',
 'pay type': 'standard',
 'components': ['mdb', 'monitoring'],
 'all_active_time': 1795.7333333333333,
 'avg_active_time': 598.5777777777778,
 'std_active_time': 711.2607104359671,
 'number_of_timer_stops': 2,
 'percentile_25': 98.66666666666667,
 'percentile_50': 194.08333333333334,
 'percentile_75': 896.2416666666667,
 'Открыт': '2020-03-07T17:27:22.274+0000',
 'В работе': '2020-03-07T19:00:28.720+0000',
 'Начальный простой': 93.1,
 'help was needed': 0,
 'support': {'akimrx', 'vr-owl'},
 'second line': set(),
 'support number': 2,
 'second line number': 0,
 'help from second line': 0}

In [96]:
get_all_ticket_info("CLOUDSUPPORT-27897")

{'ticket_name': 'CLOUDSUPPORT-27897',
 'pay type': 'free',
 'components': ['UI', 'compute', 'iam.delete'],
 'all_active_time': 69.95000000000002,
 'avg_active_time': 34.97500000000001,
 'std_active_time': 16.325000000000006,
 'number_of_timer_stops': 1,
 'percentile_25': 34.97500000000001,
 'percentile_50': 34.97500000000001,
 'percentile_75': 34.97500000000001,
 'Открыт': '2020-03-05T14:05:06.337+0000',
 'В работе': '2020-03-05T16:52:35.951+0000',
 'Начальный простой': 167.48333333333332,
 'help was needed': 1,
 'first summonee time': 23.95,
 'summonee short groups': {'IAM & Resource Management'},
 'summonee long groups': {'IAM & Resource Management'},
 'help from not second line': 1,
 'support': {'girimkhanova'},
 'second line': set(),
 'support number': 1,
 'second line number': 0,
 'help from second line': 0}

get_all_ticket_info("CLOUDSUPPORT-25593")

get_all_ticket_info("CLOUDSUPPORT-28119")

In [36]:
%%time
data = Parallel(n_jobs=-1)(delayed(get_all_ticket_info)(ticket) for ticket in ticket_names)

CPU times: user 7min 57s, sys: 6.72 s, total: 8min 4s
Wall time: 19min 26s


In [37]:
import gc
gc.collect()

1624

In [96]:
ticket_df = pd.DataFrame(data)

In [97]:
ticket_df["support"] = ticket_df["support"].apply(lambda x: sorted(list(x)))
ticket_df["components"] = ticket_df["components"].apply(lambda x: sorted(list(x)))
ticket_df["second line"] = ticket_df["second line"].apply(lambda x: sorted(list(x)))
ticket_df["summonee short groups"] =\
ticket_df["summonee short groups"].apply(lambda x: sorted(list(x)) if not pd.isnull(x) else [])
ticket_df["summonee long groups"] =\
ticket_df["summonee long groups"].apply(lambda x: sorted(list(x)) if not pd.isnull(x) else [])
ticket_df["components"] = ticket_df["components"].apply(lambda x: sorted(x) if len(x) > 0 
                                                        else ["no_component"])
ticket_df["summonee short groups"] =\
ticket_df["summonee short groups"].apply(lambda x: sorted(x) if len(x) > 0 
                                                        else ["no_summonee"])

In [98]:
def list_digest(strings):
    import hashlib, struct
    hash = hashlib.sha1()
    for s in strings:
        hash.update(struct.pack("I", len(s)))
        hash.update(s.encode('utf-8'))
    return hash.hexdigest()

In [99]:
ticket_df["components_hashed"] = ticket_df["components"].apply(lambda x: list_digest(x))
ticket_df["summonee_short_groups_hashed"] =\
ticket_df["summonee short groups"].apply(lambda x: list_digest(x))

In [100]:
def update_column(col):
    if "Открыт" == col:
        return "ticket_opened_time"
    if "В работе" == col:
        return "ticket_in_work_time"
    if "Начальный простой" == col:
        return "initial_not_working_time"
    return "_".join(col.split())

In [101]:
ticket_df.columns = [update_column(x) for x in ticket_df.columns]

In [102]:
ticket_df.replace(np.nan, 0, inplace=True)

In [ ]:
ticket_df.columns

In [104]:
ticket_df.columns = ['ticket_name', 'pay_type', 'components', 'all_active_time',
       'avg_active_time', 'std_active_time', 'number_of_timer_stops',
       'percentile_25', 'percentile_50', 'percentile_75',
       'ticket_opened_time', 'ticket_in_work_time', 'initial_not_working_time',
       'help_was_needed', 'support', 'second_line', 'support_number',
       'second_line_number', 'help_from_second_line', 'first_summonee_time',
       'summonee_short_groups', 'summonee_long_groups',
       'help_from_not_second_line', 'components_hashed',
       'summonee_short_groups_hashed']

In [105]:
ticket_df["pay_type"].replace(0, "no_type", inplace=True)

In [106]:
ticket_df.to_csv("ticket_info.csv", index=False)

In [107]:
lib.save_table("ticket_info", 
           "//home/cloud_analytics/support_tables",
           ticket_df, schema={"components":'list', "support":"list",
                              "second_line":'list',
                              "summonee_short_groups":"list", 
                              "summonee_long_groups":"list",
                              "ticket_opened_time": "datetime",
                              "ticket_in_work_time": "datetime"} )

In [108]:
ticket_df

,ticket_name,pay_type,components,all_active_time,avg_active_time,std_active_time,number_of_timer_stops,percentile_25,percentile_50,percentile_75,...,second_line,support_number,second_line_number,help_from_second_line,first_summonee_time,summonee_short_groups,summonee_long_groups,help_from_not_second_line,components_hashed,summonee_short_groups_hashed
0,CLOUDSUPPORT-4284,no_type,[vpc],596.966667,149.241667,181.837740,3,18.783333,63.283333,279.700000,...,[],2,0,0,0.0,[no_summonee],[],0.0,4ce7ac59ffb95a911ebefa48c2e0ec2ce5c1b489,cf5f8af6260284f68ab1a61d0e87a13cc3851948
1,CLOUDSUPPORT-4314,no_type,"[compute, docs]",35.200000,17.600000,10.850000,1,17.600000,17.600000,17.600000,...,[],1,0,0,0.0,[no_summonee],[],0.0,f6f82238cdaff3b60ff34e234206e966b5910489,cf5f8af6260284f68ab1a61d0e87a13cc3851948
2,CLOUDSUPPORT-4315,no_type,[speechkit],92.916667,46.458333,10.908333,1,46.458333,46.458333,46.458333,...,[],1,0,0,0.0,[no_summonee],[],0.0,2772ac0c8df69ca8397d8a32aff6337fdc963c11,cf5f8af6260284f68ab1a61d0e87a13cc3851948
3,CLOUDSUPPORT-4319,no_type,"[compute, квоты]",67.983333,22.661111,22.545881,2,7.283333,14.566667,33.991667,...,[],1,0,0,0.0,[no_summonee],[],0.0,3e2de5df0beb35229bb197b5807bb2d09ba506f7,cf5f8af6260284f68ab1a61d0e87a13cc3851948
4,CLOUDSUPPORT-4326,no_type,[compute],30.783333,10.261111,12.371789,2,1.558333,3.100000,15.383333,...,[],0,0,0,0.0,[no_summonee],[],0.0,bb838c67d8f617b6c5480be023bb435f6bbea14c,cf5f8af6260284f68ab1a61d0e87a13cc3851948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22385,CLOUDSUPPORT-28008,free,[vpc],1490.683333,745.341667,735.225000,1,745.341667,745.341667,745.341667,...,[],1,0,0,0.0,[no_summonee],[],0.0,4ce7ac59ffb95a911ebefa48c2e0ec2ce5c1b489,cf5f8af6260284f68ab1a61d0e87a13cc3851948
22386,CLOUDSUPPORT-27927,no_type,"[billing.antifraud, iam.delete]",1558.366667,222.623810,508.363457,6,11.475000,13.400000,25.158333,...,[],2,0,0,0.0,[no_summonee],[],0.0,baebea52fabca668f27c9d1407812536f5dcf88d,cf5f8af6260284f68ab1a61d0e87a13cc3851948
22387,CLOUDSUPPORT-28211,business,[mdb],224.033333,224.033333,0.000000,0,224.033333,224.033333,224.033333,...,[],1,0,0,0.0,[no_summonee],[],0.0,79a196cae9b886e37d8f84cb6b42b50763920c83,cf5f8af6260284f68ab1a61d0e87a13cc3851948
22388,CLOUDSUPPORT-28056,standard,[compute.snapshot],454.750000,227.375000,224.591667,1,227.375000,227.375000,227.375000,...,[],1,0,0,0.0,[no_summonee],[],0.0,25527f8800883c15be92054c120d69287e8cb7b8,cf5f8af6260284f68ab1a61d0e87a13cc3851948
